# ADA - Homework 2

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
sns.set_context('notebook')
sns.set_style('whitegrid')

I parse the first page to get all the option fields useful to make queries later 

In [ ]:
r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?&ww_i_reportmodel=133685247')
soup = BeautifulSoup(r.text, 'html.parser')
options = {}
for option in soup.find_all('option'):
    options[option.parent.get('name')]=options.get(option.parent.get('name'),{})
    options[option.parent.get('name')][option.text]=option.get('value')
options

In [ ]:
# INPUT:
# unite_acad : categorical from options.ww_x_UNITE_ACAD.keys()
# periode_acad : categorical from options.ww_x_PERIODE_ACAD.keys()
# periode_pedago : categorical from options.ww_x_PERIODE_PEDAGO.keys()
# hiverete : categorical from options.ww_x_HIVERETE.keys()
# 
# OUTPUT
# data : pandas dataframe containing the required data
def get_data(unite_acad,periode_acad,periode_pedago,hiverete):
    request = ('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?'
        + 'ww_x_GPS=-1'
        + '&ww_i_reportModel=133685247'
        + '&ww_i_reportModelXsl=133685270'
        + '&ww_x_UNITE_ACAD=' + options['ww_x_UNITE_ACAD'][unite_acad]
        + '&ww_x_PERIODE_ACAD=' + options['ww_x_PERIODE_ACAD'][periode_acad]
        + '&ww_x_PERIODE_PEDAGO=' + options['ww_x_PERIODE_PEDAGO'][periode_pedago]
        + '&ww_x_HIVERETE='+ options['ww_x_HIVERETE'][hiverete]
    )
    soup = BeautifulSoup(requests.get(request).text, 'html.parser')
    
    columns = [th.text for th in soup.find_all('th')[1:]]
    nColumns = len(columns)
    
    data = {}
    for name in columns:
        data[name] = []
    
    for tr in soup.find_all('tr')[2:]:
        content = tr.contents
        for i in range(nColumns):
            data[columns[i]].append(content[i].text)
    
    result = pd.DataFrame(data=data)
    result['ww_x_UNITE_ACAD']=unite_acad
    result['ww_x_PERIODE_ACAD']=periode_acad
    result['ww_x_PERIODE_PEDAGO']=periode_pedago
    result['ww_x_HIVERETE']=hiverete
    return result

A small test for the `get_data` method

In [ ]:
get_data(
    'Informatique',
    '2007-2008',
    'Bachelor semestre 1',
    "Semestre d'automne"
).head()

In [ ]:
def get_all_data(unite_acads,periode_acads,periode_pedagos,hiveretes):
    result = pd.DataFrame({})
    for unite_acad in unite_acads:
        for periode_acad in periode_acads:
            for periode_pedago in periode_pedagos:
                for hiverete in hiveretes:
                    result = pd.concat([result,get_data(unite_acad,periode_acad,periode_pedago,hiverete)])
    return result

In [ ]:
print(get_data(
    'Informatique',
    '2007-2008',
    'Bachelor semestre 1',
    "Semestre d'automne"
).shape)
print(get_data(
    'Informatique',
    '2008-2009',
    'Bachelor semestre 1',
    "Semestre d'automne"
).shape)
print(get_all_data(
    ['Informatique'],
    ['2007-2008','2008-2009'],
    ['Bachelor semestre 1'],
    ["Semestre d'automne"]
).shape)
get_all_data(
    ['Informatique'],
    ['2007-2008','2008-2009'],
    ['Bachelor semestre 1'],
    ["Semestre d'automne"]
)